# **Data Cleaning**


### **STEP 1 Load & Parse Time**


In [7]:
import pandas as pd

In [10]:
df = pd.read_csv('Plant_1_Data.csv')
df['DATE_TIME'] =pd.to_datetime(df['DATE_TIME'])
df = df.sort_values(by='DATE_TIME')

/tmp/ipython-input-2075167023.py:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DATE_TIME'] =pd.to_datetime(df['DATE_TIME'])


# **STEP 2 Initial Inspection**

In [15]:
df.info()
df.head()
df.isnull().sum()



<class 'pandas.core.frame.DataFrame'>
Index: 68778 entries, 0 to 68777
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DATE_TIME    68778 non-null  datetime64[ns]
 1   PLANT_ID     68778 non-null  int64         
 2   SOURCE_KEY   68778 non-null  object        
 3   DC_POWER     68778 non-null  float64       
 4   AC_POWER     68778 non-null  float64       
 5   DAILY_YIELD  68778 non-null  float64       
 6   TOTAL_YIELD  68778 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 4.2+ MB


,0
DATE_TIME,0
PLANT_ID,0
SOURCE_KEY,0
DC_POWER,0
AC_POWER,0
DAILY_YIELD,0
TOTAL_YIELD,0


# **STEP 3️ Remove Duplicate Records**

In [16]:
df = df.drop_duplicates()


# **STEP 4️ Remove Identifier Columns (Not Useful for Forecasting)**

In [17]:
df = df.drop(columns=['PLANT_ID', 'SOURCE_KEY'])


# **STEP 5️ Handle Missing Values (If Any)**

In [18]:
df['AC_POWER'] = df['AC_POWER'].fillna(0)
df['DC_POWER'] = df['DC_POWER'].fillna(0)


# **STEP 6️ Remove Physically Impossible Values**

In [19]:
df = df[(df['AC_POWER'] >= 0) & (df['DC_POWER'] >= 0)]


# **STEP 7️ Remove Data Leakage Columns**

In [20]:
df = df.drop(columns=['DAILY_YIELD', 'TOTAL_YIELD'])


# **STEP 8️ Remove Extreme Outliers (Sensor Spikes)**

In [21]:
ac_limit = df['AC_POWER'].quantile(0.99)
dc_limit = df['DC_POWER'].quantile(0.99)

df = df[(df['AC_POWER'] <= ac_limit) & (df['DC_POWER'] <= dc_limit)]


# **STEP 9 Final Consistency Check**

In [23]:
df.describe()


,DATE_TIME,DC_POWER,AC_POWER
count,68088,68088.000000,68088.000000
mean,2020-06-01 08:03:16.884913664,3044.791129,297.804381
min,2020-05-15 00:00:00,0.000000,0.000000
25%,2020-05-24 00:45:00,0.000000,0.000000
50%,2020-06-01 14:37:30,371.250000,35.887500
75%,2020-06-09 20:45:00,6205.294643,607.932143
max,2020-06-17 23:45:00,12899.000000,1257.812500
std,NaN,3925.175384,383.601298


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68088 entries, 0 to 68777
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   DATE_TIME  68088 non-null  datetime64[ns]
 1   DC_POWER   68088 non-null  float64       
 2   AC_POWER   68088 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 2.1 MB


# **STEP 11 Save Cleaned Dataset**

In [26]:
df.to_csv("/content/solar_cleaned.csv", index=False)


# **🔹 FEATURE ENGINEERING**

# **STEP 1️ Define Target**

In [27]:
target = 'AC_POWER'


# **STEP 2️ Create Time-Based Features**

In [28]:
df['hour'] = df['DATE_TIME'].dt.hour
df['day'] = df['DATE_TIME'].dt.day
df['month'] = df['DATE_TIME'].dt.month
df['dayofweek'] = df['DATE_TIME'].dt.dayofweek


# **STEP 3️ Create Lag Features (MOST IMPORTANT)**
**Lag = past power values.**

In [29]:
df['ac_lag_1'] = df['AC_POWER'].shift(1)
df['ac_lag_2'] = df['AC_POWER'].shift(2)
df['ac_lag_24'] = df['AC_POWER'].shift(24)

df['dc_lag_1'] = df['DC_POWER'].shift(1)
df['dc_lag_2'] = df['DC_POWER'].shift(2)
df['dc_lag_24'] = df['DC_POWER'].shift(24)


# **STEP 4️ Rolling Statistics (Trend Learning)**

In [30]:
df['ac_roll_mean_3'] = df['AC_POWER'].rolling(window=3).mean()
df['ac_roll_mean_6'] = df['AC_POWER'].rolling(window=6).mean()

df['dc_roll_mean_3'] = df['DC_POWER'].rolling(window=3).mean()
df['dc_roll_mean_6'] = df['DC_POWER'].rolling(window=6).mean()


# **STEP 5️ Drop NaNs (Expected)**

In [32]:
df = df.dropna().reset_index(drop=True)


# **STEP 6️ Final Feature Check**

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68064 entries, 0 to 68063
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE_TIME       68064 non-null  datetime64[ns]
 1   DC_POWER        68064 non-null  float64       
 2   AC_POWER        68064 non-null  float64       
 3   hour            68064 non-null  int32         
 4   day             68064 non-null  int32         
 5   month           68064 non-null  int32         
 6   dayofweek       68064 non-null  int32         
 7   ac_lag_1        68064 non-null  float64       
 8   ac_lag_2        68064 non-null  float64       
 9   ac_lag_24       68064 non-null  float64       
 10  dc_lag_1        68064 non-null  float64       
 11  dc_lag_2        68064 non-null  float64       
 12  dc_lag_24       68064 non-null  float64       
 13  ac_roll_mean_3  68064 non-null  float64       
 14  ac_roll_mean_6  68064 non-null  float64       
 15  dc

In [36]:
df.describe()


,DATE_TIME,DC_POWER,AC_POWER,hour,day,month,dayofweek,ac_lag_1,ac_lag_2,ac_lag_24,dc_lag_1,dc_lag_2,dc_lag_24,ac_roll_mean_3,ac_roll_mean_6,dc_roll_mean_3,dc_roll_mean_6
count,68064,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000,68064.000000
mean,2020-06-01 08:12:04.982369536,3045.864751,297.909390,11.568994,15.756523,5.518953,2.982355,297.909390,297.909390,297.909390,3045.864751,3045.864751,3045.864751,297.909390,297.909390,3045.864751,3045.864751
min,2020-05-15 00:15:00,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2020-05-24 00:45:00,0.000000,0.000000,6.000000,9.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2020-06-01 14:45:00,372.937500,36.040179,12.000000,16.000000,6.000000,3.000000,36.040179,36.040179,36.040179,372.937500,372.937500,372.937500,36.722024,36.765179,379.931548,380.395833
75%,2020-06-09 20:45:00,6207.392857,608.061607,18.000000,23.000000,6.000000,5.000000,608.061607,608.061607,608.061607,6207.392857,6207.392857,6207.392857,610.663244,612.877827,6233.464286,6256.438988
max,2020-06-17 23:45:00,12899.000000,1257.812500,23.000000,31.000000,6.000000,6.000000,1257.812500,1257.812500,1257.812500,12899.000000,12899.000000,12899.000000,1253.676190,1251.469643,12858.285713,12835.511905
std,NaN,3925.450850,383.628153,6.894343,8.548137,0.499644,2.058750,383.628153,383.628153,383.628153,3925.450850,3925.450850,3925.450850,381.766030,380.959599,3906.267588,3897.960659


# **STEP 7️ Save Feature-Engineered Data**

In [38]:
df.to_csv("/content/solar_features.csv", index=False)


In [39]:
import os
os.listdir("/content")


['.config',
 'solar_cleaned.csv',
 'solar_features.csv',
 '.ipynb_checkpoints',
 'Plant_1_Data.csv',
 'sample_data']

In [41]:
from google.colab import files

files.download("/content//solar_features.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>